In [ ]:
!pip install pyspark

## PySpark - Instalando a bilbioteca

In [2]:
!pip install findspark

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
df = spark.sql('''select 'Sucesso total, estamos online' as hello''')
df.show()

+--------------------+
|               hello|
+--------------------+
|Sucesso total, es...|
+--------------------+



In [10]:
#importando libraries
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F
from functools import reduce

# Data manipulation usando spark #

In [13]:
df = spark.read.csv("/content/banklist.csv", sep=',', inferSchema =True, header = True)

print('df.count :', df.count(), " numero de linhas")
print('df.col ct :', len(df.columns), " numero de colunas")
print('df.columns :', df.columns)

df.count : 561  numero de linhas
df.col ct : 6  numero de colunas
df.columns : ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


# Usando SQL no PySpark #

In [15]:
df.createOrReplaceTempView("banklist")

df_check = spark.sql('''select `Bank Name`, City, `Closing Date` from banklist''')
df_check.show(6, truncate=False)

+--------------------------------+-------------+------------+
|Bank Name                       |City         |Closing Date|
+--------------------------------+-------------+------------+
|The First State Bank            |Barboursville|3-Apr-20    |
|Ericson State Bank              |Ericson      |14-Feb-20   |
|City National Bank of New Jersey|Newark       |1-Nov-19    |
|Resolute Bank                   |Maumee       |25-Oct-19   |
|Louisa Community Bank           |Louisa       |25-Oct-19   |
|The Enloe State Bank            |Cooper       |31-May-19   |
+--------------------------------+-------------+------------+
only showing top 6 rows



# DataFrame operaçoes basicas #

In [16]:
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                NULL|   NULL|NULL|31685.68449197861|                 NULL|        NULL|
| stddev|                NULL|   NULL|NULL|16446.65659309965|                 NULL|        NULL|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [17]:
df.describe('City', 'ST').show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   NULL|NULL|
| stddev|   NULL|NULL|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



#  count, columns e schema #

In [23]:
print('Total de Linhas:', df.count())
print('Total de Colunas:', len(df.columns))
print('Colunas:', df.columns)
print('Tipo de Dados:', df.dtypes)
print('Schema:', df.schema)

Total de Linhas: 561
Total de Colunas: 6
Colunas: ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipo de Dados: [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema: StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


In [24]:
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



# Removendo Duplicatas #

In [25]:
df = df.dropDuplicates()
print('df.count:',df.count())
print('df.columns',df.columns)

df.count: 561
df.columns ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


# select colunas especificas

In [28]:
df2 = df.select(*['Bank Name', 'City'])
df2.show(5)

+--------------------+--------+
|           Bank Name|    City|
+--------------------+--------+
| First Bank of Idaho| Ketchum|
|Amcore Bank, Nati...|Rockford|
|        Venture Bank|   Lacey|
|First State Bank ...|   Altus|
|Valley Capital Ba...|    Mesa|
+--------------------+--------+
only showing top 5 rows



# Select multiplas colunas

In [30]:
col_l = list(set(df.columns) - {'CERT','ST'})
df2 = df.select(*col_l)
df2.show(5)

+------------+--------+--------------------+---------------------+
|Closing Date|    City|           Bank Name|Acquiring Institution|
+------------+--------+--------------------+---------------------+
|   24-Apr-09| Ketchum| First Bank of Idaho|      U.S. Bank, N.A.|
|   23-Apr-10|Rockford|Amcore Bank, Nati...|          Harris N.A.|
|   11-Sep-09|   Lacey|        Venture Bank| First-Citizens Ba...|
|   31-Jul-09|   Altus|First State Bank ...|         Herring Bank|
|   11-Dec-09|    Mesa|Valley Capital Ba...| Enterprise Bank &...|
+------------+--------+--------------------+---------------------+
only showing top 5 rows



# Rename colunas

In [33]:
df2 = df \
  .withColumnRenamed('Bank Name'  ,'bank_name') \
  .withColumnRenamed('Acquiring Institution'  ,'acq_institution') \
  .withColumnRenamed('Closing Date'   ,'closing_date') \
  .withColumnRenamed('ST' ,'state') \
  .withColumnRenamed('CERT',  'cert') #\

df2.show(5)

+--------------------+--------+-----+-----+--------------------+------------+
|           bank_name|    City|state| cert|     acq_institution|closing_date|
+--------------------+--------+-----+-----+--------------------+------------+
| First Bank of Idaho| Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+-----+-----+--------------------+------------+
only showing top 5 rows



# Add colunas

In [34]:
df2 = df.withColumn('State', col('ST'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-----+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|State|
+--------------------+--------+---+-----+---------------------+------------+-----+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|   ID|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|   IL|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|   WA|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|   OK|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|   AZ|
+--------------------+--------+---+-----+---------------------+------------+-----+
only showing top 5 rows



# Adicionar Colunas constante

In [35]:
df2 = df.withColumn('country', lit('US'))
df2.show(5)

+--------------------+--------+---+-----+---------------------+------------+-------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|country|
+--------------------+--------+---+-----+---------------------+------------+-------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|     US|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|     US|
|        Venture Bank|   Lacey| WA|22868| First-Citizens Ba...|   11-Sep-09|     US|
|First State Bank ...|   Altus| OK| 9873|         Herring Bank|   31-Jul-09|     US|
|Valley Capital Ba...|    Mesa| AZ|58399| Enterprise Bank &...|   11-Dec-09|     US|
+--------------------+--------+---+-----+---------------------+------------+-------+
only showing top 5 rows



#Drop colunas

In [36]:
df2 = df.drop('CERT')
df2.show(5)

+--------------------+--------+---+---------------------+------------+
|           Bank Name|    City| ST|Acquiring Institution|Closing Date|
+--------------------+--------+---+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| WA| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus| OK|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| AZ| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---+---------------------+------------+
only showing top 5 rows



#Drop mais de uma coluna

In [37]:
df2 = df.drop(*['CERT','ST'])
df2.show(5)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
|        Venture Bank|   Lacey| First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|   Altus|         Herring Bank|   31-Jul-09|
|Valley Capital Ba...|    Mesa| Enterprise Bank &...|   11-Dec-09|
+--------------------+--------+---------------------+------------+
only showing top 5 rows



In [38]:
df2 = reduce(DataFrame.drop, ['CERT','ST'], df)
df2.show(2)

+--------------------+--------+---------------------+------------+
|           Bank Name|    City|Acquiring Institution|Closing Date|
+--------------------+--------+---------------------+------------+
| First Bank of Idaho| Ketchum|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---------------------+------------+
only showing top 2 rows



#Filtrar dados

In [40]:
#valores iguais
df2 = df.where(df['ST'] == 'NE')

#entre valores
df3 = df.where(df['CERT'].between('1000','2000'))

#esta dentro de multiplos valores
df4 = df.where(df['ST'].isin('NE','IL'))

print('df.count :', df.count())
print('df2.count :', df2.count())
print('df3.count :', df3.count())
print('df4.count :', df4.count())

df.count : 561
df2.count : 4
df3.count : 9
df4.count : 73


#Filtrar dados usando operadores logicos

In [43]:
df2 = df.where((df['ST'] == 'NE') & (df['City'] == 'Ericson'))
df2.show(2)

+------------------+-------+---+-----+---------------------+------------+
|         Bank Name|   City| ST| CERT|Acquiring Institution|Closing Date|
+------------------+-------+---+-----+---------------------+------------+
|Ericson State Bank|Ericson| NE|18265| Farmers and Merch...|   14-Feb-20|
+------------------+-------+---+-----+---------------------+------------+



#Replace valores no dataframe

In [44]:
#Antes do replace
df.show(2)
#Pós replace
print('alterar valor 7 para 17')
df.na.replace(7,17).show(2)

+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---+-----+---------------------+------------+
only showing top 2 rows

alterar valor 7 para 17
+--------------------+--------+---+-----+---------------------+------------+
|           Bank Name|    City| ST| CERT|Acquiring Institution|Closing Date|
+--------------------+--------+---+-----+---------------------+------------+
| First Bank of Idaho| Ketchum| ID|34396|      U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|Rockford| IL| 3735|          Harris N.A.|   23-Apr-10|
+--------------------+--------+---+-----+---------------------+------------+
only showing top 2 rows

